# Анализ текстовых данных - Natural Language Processing, NLP

## Предварительная обработка

### Векторизация

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df = 1)
print (vectorizer)

CountVectorizer()


### Векторизация корпуса из двух документов
Корпус задан как список строк

In [2]:
test_content = ['How to format my hard disk', 'Hard disk format problems']
# векторизация
X = vectorizer.fit_transform(test_content)
print ('shape -> (число документов, длина словаря)')
print(X.shape)
print('Результат векторизации:')
print(X)

shape -> (число документов, длина словаря)
(2, 7)
Результат векторизации:
  (0, 3)	1
  (0, 6)	1
  (0, 1)	1
  (0, 4)	1
  (0, 2)	1
  (0, 0)	1
  (1, 1)	1
  (1, 2)	1
  (1, 0)	1
  (1, 5)	1


Получим словарь корпуса (по алфавиту)

In [3]:
vectorizer.get_feature_names_out()

array(['disk', 'format', 'hard', 'how', 'my', 'problems', 'to'],
      dtype=object)

## Пример с корпусом текстов сообщений

Используем учебный датасет https://www.kaggle.com/crawford/20-newsgroups

В нем около 18 000 документов (посты в группах), отнесенных к 20 группам-категориям.

Можно легко выбирать отдельные категории (см.далее).

In [4]:
from sklearn.datasets import fetch_20newsgroups

Как можно взять часть данных? Явно указать названия категорий.

In [5]:
categories = ['comp.graphics','comp.windows.x']
dataset = fetch_20newsgroups(subset='all', categories=categories, shuffle=True, random_state=4)
print("%d documents" % len(dataset.data))
print("%d categories" % len(dataset.target_names))

1961 documents
2 categories


Возьмем первые 50 документов

In [6]:
posts = dataset.data[:50]
# полезно посмотреть на исходные тексты
posts[:5]

['From: joshuaf@yang.earlham.edu\nSubject: Re: TIFF -> Anything?!\nOrganization: Earlham College, Richmond, Indiana\nLines: 15\n\nIn article <1993Apr23.033843.26854@spartan.ac.BrockU.CA>, tmc@spartan.ac.BrockU.CA (Tim Ciceran) writes:\n> There is a program called Graphic Workshop you can FTP from\n> wuarchive.  The file is in the msdos/graphics directory and\n> is called "grfwk61t.zip."  This program should od everthing\n> you need.\n> \n> -- \n> \n> TMC\n> (tmc@spartan.ac.BrockU.ca)\n\n\nTHANKS!  It did work, and it is just what I needed thanks...\n\nJoshuaf\n',
 'From: dealy@narya.gsfc.nasa.gov (Brian Dealy - CSC)\nSubject: Re: Monthly Question about XCopyArea() and Expose Events\nOrganization: NASA/Goddard Space Flight Center\nLines: 43\nDistribution: world\nNNTP-Posting-Host: narya.gsfc.nasa.gov\nOriginator: dealy@narya.gsfc.nasa.gov\n\n\n|> (2nd posting of the question that just doesn\'t seem to get answered)\n|> \n|> Suppose you have an idle app with a realized and mapped Window 

### Векторизация
Выведем результаты - число документов и длину словаря, которая определяет длину полученных векторов

In [7]:
vectorizer = CountVectorizer(min_df = 1)
X_train = vectorizer.fit_transform(posts)
# число сообщений, длина словаря
num_samples, num_features = X_train.shape
print ('ns= %d, nf= %d' % (num_samples, num_features))

ns= 50, nf= 4595


Вывод словаря для выбранных документов - видно, что есть токены, не имеющие особого смысла

In [8]:
print (vectorizer.get_feature_names_out())

['00' '000' '000005102000' ... 'zoyd' 'zurich' 'zyeh']


### Поиск "похожих" документов
Зададим произвольный текст вопроса и векторизуем его на основе имеющегося словаря

In [30]:
# векторизация сообщения-вопроса
new_post = 'Who is from Richmond Indiana or from Zurich'
new_post_vec = vectorizer.transform([new_post])
print (new_post_vec)

  (0, 1812)	2
  (0, 2175)	1
  (0, 2260)	1
  (0, 2976)	1
  (0, 3530)	1
  (0, 4391)	1
  (0, 4593)	1


Все слова есть в словаре -> можно найти "похожие" документы!

### Подготовка для вычисления расстояний

Построим метрику напрямую:

In [32]:
# функция для евклидова расстояния
import scipy as sp
def dist_eucl(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())


Поиск ближайшего сообщения с промежуточной печатью

In [33]:
import sys
# поиск ближайшего сообщения
best_post = []
best_dist = sys.maxsize
best_i = 0
for i in range(0, num_samples):
  post_vec = X_train.getrow(i)
  d = dist_eucl(post_vec, new_post_vec)
  print('dist=%.2f'%d)
  if d < best_dist:
    best_dist = d
    best_i = i
print ('best post is %i dist=%.2f' % (best_i, best_dist))


dist=11.66
dist=49.07
dist=12.00
dist=18.89
dist=37.58
dist=15.81
dist=19.05
dist=9.38
dist=12.49
dist=12.33
dist=8.00
dist=36.61
dist=18.08
dist=25.42
dist=34.50
dist=29.41
dist=14.14
dist=13.45
dist=12.33
dist=19.87
dist=10.54
dist=774.36
dist=540.17
dist=8.31
dist=19.47
dist=30.84
dist=19.70
dist=16.43
dist=23.28
dist=22.36
dist=13.45
dist=31.13
dist=40.68
dist=18.36
dist=75.22
dist=20.22
dist=10.86
dist=142.25
dist=33.06
dist=8.37
dist=19.54
dist=7.35
dist=28.88
dist=21.73
dist=27.15
dist=10.30
dist=9.06
dist=25.36
dist=28.34
dist=24.62
best post is 41 dist=7.35


In [35]:
print(posts[best_i])

From: queloz@bernina.ethz.ch (Ronald Queloz)
Subject: Hypercard for UNIX
Organization: Swiss Federal Institute of Technology (ETH), Zurich, CH
Lines: 10

Hi netlanders,

Does anybody know if there is something like Macintosh Hypercard for any UNIX 
platform?


Thanks in advance


Ron.



Нормируем векторы сообщений к длине=1 и изменим функцию для расстояния

In [36]:
# переход к расстоянию между нормированными векторами
def dist_norm(v1, v2):
    v1_norm = v1/sp.linalg.norm(v1.toarray())
    v2_norm = v2/sp.linalg.norm(v2.toarray())
    delta = v1_norm - v2_norm
    return sp.linalg.norm(delta.toarray())

In [37]:
# поиск ближайшего сообщения
best_post = None
best_dist = sys.maxsize
best_i = None
for i in range(0, num_samples):
  post_vec = X_train.getrow(i)
  d = dist_norm(post_vec, new_post_vec)
  if i%2==0:
    print('dist=%.2f'%d)
  if d < best_dist:
    best_dist = d
    best_i = i
print ('best post is %i dist=%.2f' % (best_i, best_dist))

dist=1.22
dist=1.38
dist=1.37
dist=1.34
dist=1.28
dist=1.35
dist=1.38
dist=1.35
dist=1.30
dist=1.34
dist=1.35
dist=1.30
dist=1.39
dist=1.34
dist=1.35
dist=1.33
dist=1.36
dist=1.36
dist=1.35
dist=1.38
dist=1.34
dist=1.29
dist=1.33
dist=1.28
dist=1.33
best post is 0 dist=1.22


In [39]:
print(posts[best_i])

From: joshuaf@yang.earlham.edu
Subject: Re: TIFF -> Anything?!
Organization: Earlham College, Richmond, Indiana
Lines: 15

In article <1993Apr23.033843.26854@spartan.ac.BrockU.CA>, tmc@spartan.ac.BrockU.CA (Tim Ciceran) writes:
> There is a program called Graphic Workshop you can FTP from
> wuarchive.  The file is in the msdos/graphics directory and
> is called "grfwk61t.zip."  This program should od everthing
> you need.
> 
> -- 
> 
> TMC
> (tmc@spartan.ac.BrockU.ca)


THANKS!  It did work, and it is just what I needed thanks...

Joshuaf



### Как влияют стоп-слова?

Убираем стоп-слова

In [40]:
vectorizer = CountVectorizer(min_df = 1, stop_words = 'english')

sorted(vectorizer.get_stop_words())[:15]

['a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although']

In [41]:
X_train = vectorizer.fit_transform(posts)
# число сообщений, число слов
num_samples, num_features = X_train.shape
print ('ns= %d, nf= %d' % (num_samples, num_features))

ns= 50, nf= 4360


In [42]:
new_post_vec = vectorizer.transform([new_post])

In [43]:
# поиск ближайшего сообщения
best_post = None
best_dist = sys.maxsize
best_i = None
for i in range(0, num_samples):
  post_vec = X_train.getrow(i)
  d = dist_norm(post_vec, new_post_vec)
  # проверочная печать
  print('dist=%.2f'%d)

  if d < best_dist:
    best_dist = d
    best_i = i
print ('best post is %i dist=%.2f' % (best_i, best_dist))


dist=1.33
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.37
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.35
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
dist=1.41
best post is 0 dist=1.33


# Библиотека nltk - Natural Language Tool Kit
https://www.nltk.org/

Стоп-слова для русского языка

In [44]:
import nltk
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('russian')
print(stopwords)


['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Стемминг с помощью nltk

In [45]:
import nltk.stem
s=nltk.stem.SnowballStemmer('english')

s.stem('imaging')

'imag'

In [46]:
s.stem('image')

'imag'

In [47]:
s.stem('imagination')

'imagin'

In [48]:
s=nltk.stem.SnowballStemmer('russian')

s.stem('домик')

'домик'

In [49]:
s.stem('дома')

'дом'

# Векторизация TF-IDF
Простой пример для прямого вычисления векторов весов в корпусе из трех токенизированных документов

In [51]:
import numpy as np

def tfidf(t, d, D):
    tf = float(d.count(t)) / sum(d.count(w) for w in set(d))
    idf = np.lib.scimath.log(float(len(D)) / (len([doc for doc in D if t in doc])))
    return tf * idf
# документы
a1, a2, a3 = ['a'], ['a', 'b', 'b'], ['a', 'b', 'c']
# корпус
D = [a1, a2, a3]
print (tfidf('a', a1, D))
print (tfidf('a', a2, D))
print (tfidf('a', a3, D))
print (tfidf('b', a2, D))
print (tfidf('b', a3, D))
print (tfidf('c', a3, D))


0.0
0.0
0.0
0.27031007207210955
0.13515503603605478
0.3662040962227032
